https://www.python-course.eu/threads.php

In [1]:
import time
from threading import Thread

def sleeper(i):
    print ("\nthread %d sleeps for 5 seconds" % i)
    time.sleep(5)
    print ("\nthread %d woke up" % i)

for i in range(10):
    t = Thread(target=sleeper, args=(i,))
    t.start()


thread 0 sleeps for 5 seconds

thread 1 sleeps for 5 seconds

thread 2 sleeps for 5 seconds

thread 3 sleeps for 5 seconds

thread 4 sleeps for 5 seconds

thread 5 sleeps for 5 seconds

thread 6 sleeps for 5 seconds

thread 7 sleeps for 5 seconds

thread 8 sleeps for 5 seconds

thread 9 sleeps for 5 seconds

thread 0 woke up

thread 5 woke up
thread 2 woke up


thread 7 woke up

thread 1 woke up

thread 3 woke up
thread 4 woke up


thread 6 woke up

thread 8 woke up
thread 9 woke up



## prime-factoring

In [2]:
import threading 
 
class PrimeNumber(threading.Thread): 
  def __init__(self, number): 
    threading.Thread.__init__(self) 
    self.Number = number
 
  def run(self): 
    counter = 2 
    while counter*counter <= self.Number: 
      if self.Number % counter == 0: 
        print ("\n %d = %d * %d [not prime]" % ( self.Number, counter, self.Number / counter) )
        return 
      counter += 1 
    print (f"\n {self.Number} [prime]")
        
threads = [] 
while True: 
    numbers = [int(n) for n in input("numbers (space delimited, enter 0 to exit): ").split()]
    if len(numbers) == 1 and numbers[0] < 2: 
        break 
 
    for n in numbers:
        if n < 2: continue
        thread = PrimeNumber(n) 
        threads += [thread] 
        thread.start() 

for x in threads: 
    x.join()

numbers (space delimited, enter 0 to exit): 3 23 21 121 10000000000000000001

 3 [prime]

 23 [prime]

 21 = 3 * 7 [not prime]

 121 = 11 * 11 [not prime]

 10000000000000000001 = 11 * 909090909090909056 [not prime]
numbers (space delimited, enter 0 to exit): 3 23 21 121 100000000001

 3 [prime]

 23 [prime]

 21 = 3 * 7 [not prime]

 121 = 11 * 11 [not prime]

 100000000001 = 11 * 9090909091 [not prime]
numbers (space delimited, enter 0 to exit): 1000000000011

 1000000000011 = 3 * 333333333337 [not prime]
numbers (space delimited, enter 0 to exit): 0


## Thread with return value

https://stackoverflow.com/questions/6893968/how-to-get-the-return-value-from-a-thread-in-python

In [26]:
def prime_factor(n):
    if n < 2:
        return None
    counter = 2 
    while counter*counter <= n: 
      if n % counter == 0: 
        factor = int(n / counter)
        print ("%d = %d * %d [not prime]" % ( n, counter, factor) )
        return (counter, factor)
      counter += 1 
    print (f"{n} [prime]")
    return (n,)

In [27]:
prime_factor(11), prime_factor(1), prime_factor(121)

11 [prime]
121 = 11 * 11 [not prime]


((11,), None, (11, 11))

In [28]:
from threading import Thread

class PrimeNumber(Thread):
    def __init__(self, group=None, target=None, name=None, args=(), kwargs=None, *, daemon=None):
        Thread.__init__(self, group, target, name, args, kwargs, daemon=daemon)
        self._return = None

    def run(self):
        if self._target is not None:
            self._return = self._target(*self._args, **self._kwargs)

    def join(self):
        Thread.join(self)
        return self._return

In [29]:
_n2t = {}
while True: 
    numbers = [int(n) for n in input("numbers (space delimited, enter 0 to exit): ").split()]
    if len(numbers) == 1 and numbers[0] < 2: 
        break 
 
    for n in numbers:
        if n < 2: 
            _n2t[n] = None
            continue
        if n in _n2t:
            continue
            
        thread = PrimeNumber(target=prime_factor, args=(n,))
        _n2t[n] = thread
        thread.start() 

_n2p = {}
for n,t in _n2t.items():
    if t:
        _n2p[n] = t.join()
        
print("Primes are ...")
for n, factors in _n2p.items():
    if len(factors) < 2:
        print(n)

numbers (space delimited, enter 0 to exit): 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 
2 [prime]
3 [prime]
4 = 2 * 2 [not prime]
5 [prime]6 = 2 * 3 [not prime]

7 [prime]
8 = 2 * 4 [not prime]
9 = 3 * 3 [not prime]
10 = 2 * 5 [not prime]
11 [prime]
12 = 2 * 6 [not prime]
13 [prime]
14 = 2 * 7 [not prime]
15 = 3 * 5 [not prime]
16 = 2 * 8 [not prime]
17 [prime]
18 = 2 * 9 [not prime]
19 [prime]
numbers (space delimited, enter 0 to exit): 0
Primes are ...
2
3
5
7
11
13
17
19


In [21]:
for i in range(1, 20): 
    print(i, end =" ") 

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 

### scan network

#### ping host

In [32]:
import os, re, threading

RECEIVED_PACKAGES = re.compile(r"(\d) received")

class ip_check(threading.Thread):
   def __init__ (self,ip):
      threading.Thread.__init__(self)
      self.ip = ip
      self.__successful_pings = -1
   def run(self):
      ping_out = os.popen("ping -q -c2 "+self.ip,"r")
      while True:
        line = ping_out.readline()
        if not line: break
        n_received = re.findall(RECEIVED_PACKAGES,line)
        if n_received:
           self.__successful_pings = int(n_received[0])
   def status(self):
      if self.__successful_pings == 0:
         return "no response"
      elif self.__successful_pings == 1:
         return "alive, but 50 % package loss"
      elif self.__successful_pings == 2:
         return "alive"
      else:
         return "shouldn't occur"

# use $ ifconfig to get ip addr range of local network
check_results = []
for suffix in range(1,10):
   ip_ = "192.168.1." + str(suffix)
   current = ip_check(ip_)
   check_results.append(current)
   current.start()

for el in check_results:
   el.join()
   print ("Status from ", el.ip, " is",el.status())

Status from  192.168.1.1  is alive
Status from  192.168.1.2  is alive
Status from  192.168.1.3  is alive
Status from  192.168.1.4  is no response
Status from  192.168.1.5  is no response
Status from  192.168.1.6  is no response
Status from  192.168.1.7  is no response
Status from  192.168.1.8  is no response
Status from  192.168.1.9  is no response


#### scan port

In [38]:
def scan_port(addr):
    import platform
    import socket
    os_type = platform.system() #  "Windows"

    _dict_ports = {
        "Windows" : [135, 137, 138, 139, 445], 
        "Linux" : [20, 21, 22, 23, 25, 80, 111, 443, 445, 631, 993, 995], 
        "Mac" : [22, 445, 548, 631]
    }
    
    open_ports = []
    
    try:
        ports = _dict_ports[os_type]
    except:
        return open_ports
    
    for port in ports:
        socket_obj = socket.socket(socket.AF_INET,socket.SOCK_STREAM)
        socket.setdefaulttimeout(1)
        result = socket_obj.connect_ex((addr,port))
        socket_obj.close()
        if result == 0:
            print(f"{addr}:{port} - open")
            open_ports.append(port)
            
    return open_ports

In [39]:
scan_port("192.168.1.3")

192.168.1.3:22 - open
192.168.1.3:111 - open
192.168.1.3:445 - open


[22, 111, 445]